In [ ]:
import numpy as np
import scipy.optimize as opt
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import scipy.linalg as la
import numpy as np
from numpy.linalg import matrix_rank

import sympy as sym
from sympy import *
from sympy import symbols, Matrix, Transpose
from sympy import lambdify

import math
import cmath
from math import *

import mpmath as mp
from math import sqrt
from matplotlib import rcParams



In [ ]:
# Parameter values

# Iteration parameters
S = 100 # number of periods in the shooting
niter=200 # number of possible trials of the intial value
epsilon=.00005 #convergence criterion

# Model Parameters
beta = 0.933 # Discount factor
theta = 0.3 # Share of capital in output rents
alpha = 3.2 # disutility from labor parameter
delta = 0.035 # Capital depreciation rate
T =5*24 # Total number of hours owned in a week


In [ ]:

gamma = 1.014 # TFP to the power 1/(1-theta) growth rate. Long run growth rate of per capita income 
gA_ast = gamma**(1-theta) # = (1+g) in our model. TFP growth rate
n_ast = 1.014 # population growth rate. n_ast = (1+n) in our model

chi_ast = 0.176 # long-run government share
tau_ast = 0.098 # long-run capital tax


N0 = 10900104
H0 = 23.0345983*N0
k0 = 2.47**(1/(1-theta))*(H0/N0)
A0 = 1**(1-theta)


In [ ]:
# Steady State
c1 = (((gA_ast**(1/(1-theta))/beta - 1)*(1/(1-tau_ast))+ delta)*1/theta)**(1/(theta - 1)) # steady state k/h ratio
c2 = (1 - chi_ast)*c1**(theta - 1) - (gA_ast**(1/(1-theta))*n_ast - 1 + delta)  # steady state c/k ratio
c3 = c2*alpha/((1-theta)*c1**(theta-1)) # steady state (T - h)/h
h_ast = 1/(1+c3)*T
k_ast = c1*h_ast
y_ast = k_ast**theta*h_ast**(1-theta)/c1**(theta-1)
c_ast = c2*y_ast

print("[k0, k_ast, c_ast, h_ast] = ", k0, k_ast, c_ast, h_ast)

In [ ]:
# Trajectory of Exogenous Variables



        
A_t = [1, 1.04378473, 1.101607201, 1.150984455, 1.181143954, 1.137405416, 1.146534393, 1.140791121, 1.214149745, 1.236197781, 1.268104407, 1.234881423, 1.218360599, 1.243219825, 1.283024964, 1.30214835, 1.361505992, 1.364407932, 1.369758769, 1.338515507, 1.370358575, 1.299345982, 1.351457244, 1.34782411, 1.360881802, 1.37488693, 1.374290051, 1.41595458, 1.394709054, 1.301317914, 1.249561069, 1.260135895, 1.249347697, 1.25594754, 1.339332806, 1.375978959, 1.483980171, 1.55773101, 1.554106291, 1.488266754, 1.496353864]
gA = [1]*S #gA is (1+g) in our model. TFP growth rate
for i in range(0, S):
    if i < len(A_t):
        gA[i] = (A_t[i]/A_t[i-1])**(1-theta)
    else:
        gA[i] = (A_t[len(A_t)-1]/A_t[len(A_t)-2])**(1-theta)
        
    

gamma = [1]*S # TFP to the power 1/(1-theta) growth rate. Long run growth rate of per capita income 
for i in range(0, S):
    if i < len(A_t):
        gamma[i] = (A_t[i]/A_t[i-1])
    else:
        gamma[i] = (A_t[len(A_t)-1]/A_t[len(A_t)-2])
        
A = [1]*S
for i in range(0, S):
    if i < len(A_t):
        A[i] = A_t[i]**(1-theta)
    else:
        A[i] = A_t[len(A_t)-1]**(1-theta)
    


In [ ]:
# Initial Conditions for capital

k = [1]*(S)
c = [1]*(S)
h = [1]*(S)

k[0] = k0

#Initial condition for consumption
c_0_L = .05 *c_ast
c_0_H = 1.05*c_ast


#Solving for k_t+1 and c_t+1 and h_t+1 until convergence along the saddle path
niter = 500
S = 100
flag  = 0

#Start iteration
for j in range(0,niter):
    c[0] = (c_0_L+c_0_H)/2 #Initial guess for c_o
    
    def func(x): 
        return T - (k[0]/x)**(-theta)*(alpha*c[0]/(1-theta)) - x #Consumption-Leisure choice

    root = fsolve(func, 15)
    h[0] = root[0]
    
    
    #Generate paths for c and k given intitial guess for c_0
    for t in range(0,S-1):
        k[t+1] = (k[t]**theta*h[t]**(1-theta)*(1- chi_ast) +(1-delta)*k[t]-c[t])/((gamma[t])*n_ast)
        #print(k[t+1])
        
        
        if k[t+1] < 0:
            flag = 1
            #print(flag)
            break
        if k[t+1]>2*k_ast:
            flag =2
            #print(flag)
            break
            
        def cfunc(z):
            # Solves for c_t+1 and h_t+1 simultaneously
            fc = z[0]
            fh = z[1]
            f1 = T - (alpha*fc/(1-theta))*(k[t+1]/fh)**(-theta) - fh 
            f2 = fc/c[t] - (beta/gamma[t])*(1 + (1-tau_ast)*(theta*((k[t+1]/fh)**(theta-1)) - delta)) 
            return [f1, f2]
        
        sol = fsolve(cfunc, [15, 15])
        
        c[t+1] = sol[0]
        h[t+1] = sol[1]
        
        # Check for convergence
        # Update the intitial value if not convergence
        if t== S-2:
            
            if (abs(k[S-1]-k_ast)/k_ast < epsilon) and abs((c[S-1] -c_ast)/c_ast < epsilon):
                flag = 10
                print( "Converged in iteration ", j)
                break
            # Update initial guess for c
            elif (k[S-1]-k_ast)/k_ast <=epsilon:
                flag =1
            elif (k[S-1] - k_ast)/k_ast >= epsilon:
                flag =2
        
        
    if flag == 1:
        c_0_H = c[0]
        
    elif flag == 2:
        c_0_L = c[0]
        
    elif flag == 10:
        break
        


        #Ends iteration loop
        #End shooting algorithm if reached max number of iterations
    
if j == niter-1:
    print('Failure in Convergence!')


In [ ]:
t = S-2
print(k[t+1], c[t+1], h[t+1])
print([k_ast, c_ast, h_ast])
print (c[0])
print (h[0])
print(epsilon)
print((k[S-1]-k_ast)/k_ast <epsilon)
print((c[S-1]-c_ast)/c_ast <epsilon)
print((h[S-1]-h_ast)/h_ast)
print ((c[S-1]-c_ast)/c_ast)

print (c[S-1])